In [4]:
import requests
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import time
import os
print("Imported Succesfully")


Imported Succesfully


In [5]:
# Pre-Defining the Columns
columns = ["Job", "Company", "Location",
           "Pay", "Details", "Internship Link"]
sample_df = pd.DataFrame(columns=columns)

# The Base URL will be the same the skill is added to the link for finding particular interns related to a particular domain
base_url = "https://internshala.com"
skill = "Machine Learning"


In [6]:
# Pulling the Job_Title
def job_title(job_div, job_post):
    for a in job_div.find(name="div", attrs={"class": "company"}).find(name="div", attrs={"class": "profile"}).find(name="a"):
        job_post.append(a)

# Pulling the Company 
def company(job_div, job_post):
    for a in job_div.find(name="div", attrs={"class": "individual_internship_header"}).find(name="div", attrs={"class": "company"}).find(name="div", attrs={"class": "company_name"}).find(name="a"):
        job_post.append(str(a).strip())

# Pulling the location of the Company
def location(job_div, job_post):
    for a in job_div.find(name="div", attrs={"class": "individual_internship_details"}).find(name="div", attrs={"id": "location_names"}).find(name="a", attrs={"class": "location_link"}):
        job_post.append(str(a).strip())

# Pulling the Salary Details of the Specific Post
def salary(job_div, job_post):
    for stipend in job_div.find(name="div", attrs={"class": "individual_internship_details"}).find(name="div", attrs={"class": "internship_other_details_container"}).find(name="span", attrs={"class": "stipend"}):
        if isinstance(stipend, bs4.element.NavigableString):
            job_post.append(str(stipend).strip())

# Pulling the Duraation Of The Internship
def duration(job_div, job_post):
    for stipend in job_div.find(name="div", attrs={"class": "individual_internship_details"}).find(name="div", attrs={"class": "internship_other_details_container"}).find(name="span", attrs={"class": "stipend"}):
        if isinstance(stipend, bs4.element.NavigableString):
            job_post.append(str(stipend).strip())

# Getting the UrL of the Page as a clickable link into the csv file
def Get_URL_Of_page(base_url, skill):
    return base_url + "/internships/" + skill + "-internship"


# For Getting the Pages for Web Scraping
def Get_total_pages(url):
    # get total number of pages
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser", from_encoding="utf-8")
    total_pages = soup.find(name="span", attrs={"id": "total_pages"})
    return int(total_pages.text.strip())


# 
def Internship_Description_Page_Url(job_div, base_url):
    for a in job_div.find(name="div", attrs={"class": "button_container"}).find_all(name="a", attrs={"class": "view_detail_button"}, href=True):
        newvar = base_url + a['href']
        return newvar
        


# The Whole Function is For Pulling Specific Description of the Company
def extract_description(job_div, job_post):
    slink = Internship_Description_Page_Url(job_div, base_url)
    page = requests.get(slink)
    soup_desc = BeautifulSoup(page.text, "html.parser", from_encoding="utf-8")


    for div in soup_desc.find_all(name="div", attrs={"class": "section_heading heading_5_5"}):
        if any(word in div.text.strip() for word in ["job/internship", "internship", "work from home", "part time", "job"]):
            job_post.append(div.next_sibling.next_sibling.text.strip())
    job_post.append(slink)




In [7]:

def main_func(base_url, skill):
    url = Get_URL_Of_page(base_url, skill)
    total_pages = Get_total_pages(url)
    for page_number in range(total_pages):
        page = requests.get(url + "/page-" + str(page_number + 1))
        time.sleep(0.5)
        soup = BeautifulSoup(page.text, "html.parser")

        for div in soup.find_all(name="div", attrs={"class": "individual_internship"}):
            # specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1)
            # Blank List which is to be filled later
            job_post = []
            # Calling all the above functions defined 
            job_title(div, job_post)
            company(div, job_post)
            location(div, job_post)
            salary(div, job_post)
            extract_description(div, job_post)
            # appending list of job post info to dataframe at index num
            sample_df.loc[num] = job_post

    return sample_df


In [8]:
# Calling All The Functions
df_internships = main_func(base_url, skill)

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [9]:
# Displays the Datframe
df_internships

,Job,Company,Location,Pay,Details,Internship Link
1,Machine Learning,Mayand Infotech Private Limited,Work From Home,3000-4000 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/mach...
2,Engineering (Deep Learning & Computer Vision),AiBorne Tech,Work From Home,11000-15000 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/engi...
3,Validation (AI/ML/DL),Wobot Intelligence Private Limited,Work From Home,7000 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/vali...
4,Machine Learning (Computer Vision),Alphabt,Work From Home,15000-20001 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/mach...
5,Research And Development,Isourse,Delhi,15000 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/rese...
6,Data Science Research,Anurag MN,Work From Home,2000-6000 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/data...
7,Machine Learning/AI,Honeysys,Bangalore,5000 /month + Incentives,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/mach...
8,Data Science & Machine Learning,Markytics,Pune,5000-10000 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/data...
9,Data Science (Machine Learning),Techvolt Software Private Limited,Coimbatore,2000 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/data...
10,Data Science,"Kaatru, IIT Madras",Chennai,21500 /month,Selected intern's day-to-day responsibilities ...,https://internshala.com/internship/detail/data...


In [10]:
# Converting the Dataframe into a CSV
df_internships.to_csv("FinalCSV1.csv",encoding="utf-8")